***Промежуточный проект***\
**Тимонин Сергей**

### Входные данные

In [141]:
import pandas as pd
import numpy as np
import plotly as plt
import math
import seaborn as sns
from datetime import timedelta
from operator import attrgetter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")

In [87]:
customers =pd.read_csv('olist_customers_dataset_2.csv', sep = ';') # таблица с уникальными идентификаторами пользователей
orders =pd.read_csv('olist_orders_dataset.csv', sep = ',') #  таблица заказов
order_items =pd.read_csv('olist_order_items_dataset.csv', sep = ',')  # товарные позиции, входящие в заказы
orders.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


### Задача 1. Сколько у нас пользователей, которые совершили покупку только один раз?  
Под покупкой я подразумеваю ОПЛАЧЕННЫЙ (есть дата оплаты) и ДОСТАВЛЕННЫЙ (статус = delivered) заказ.  

In [8]:
order_by_cust = orders.merge(customers, how = 'left', on = 'customer_id')
order_by_cust['order_purchase_timestamp']=pd.to_datetime(order_by_cust['order_purchase_timestamp'])
order_by_cust['order_approved_at']=pd.to_datetime(order_by_cust['order_approved_at'])
order_by_cust.dropna(subset=['order_approved_at'], inplace=True) # удаляю строки с пустой датой платежа
order_by_cust['order_approved_at'].isnull().sum() # проверяю колво строк с пустой датой платежа

0

In [9]:
order_by_cust_1_time= order_by_cust[order_by_cust['order_status'] =='delivered']\
.groupby('customer_unique_id')\
.agg({'order_id':'nunique'})\
.sort_values(by = 'order_id', ascending = False)
order_by_cust_1_time

,order_id
customer_unique_id,
8d50f5eadf50201ccdcedfb9e2ac8455,15
3e43e6105506432c953e165fb2acf44c,9
6469f99c1f9dfae7733b25662e7f1782,7
ca77025e7201e3b30c44b472ff346268,7
1b6c7548a2a1f9037c1fd3ddfed95f33,7
...,...
5656a8fabc8629ff96b2bc14f8c09a27,1
56560124a5cc70119deb685775639ea7,1
5655ee330a79bd1fd1e5fb220c2436dd,1


In [10]:
# ВНИМАНИЕ ОТВЕТ
answer_1 =order_by_cust_1_time[(order_by_cust_1_time.order_id == 1)]
print('Пользователей совершивших покупку 1 раз, и покупка была доставлена:',answer_1.shape[0])

Пользователей совершивших покупку 1 раз, и покупка была доставлена: 90545


### Задача 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? 

In [11]:
# группирую данные по статусам
orders_2=orders[orders['order_status'] !='delivered']\
.groupby(['order_status','order_purchase_timestamp','order_id'], as_index = False)\
.order_id.count()

In [12]:
# меняю формат в поле с датой + создаю столбцы с месяцем и годом
orders_2['order_purchase_timestamp']=pd.to_datetime(orders_2['order_purchase_timestamp'])
orders_2['month'] = orders_2['order_purchase_timestamp'].dt.month
orders_2['year'] = orders_2['order_purchase_timestamp'].dt.year

In [13]:
# ДОПОЛНИТЕЛЬНО - сводная таблица с разбивкой по месяцам годам (можно сделать график с динамикой)
ex_2=pd.pivot_table(orders_2,
               index=["order_status"],
               values=["order_id"],
               columns=["year", "month"],
               aggfunc=[np.sum]).fillna(0)
ex_2

sum                                                        \
             order_id                                                         
year             2016        2017                                             
month              9     10    1     2     3     4     5     6     7     8    
order_status                                                                  
approved          0.0   0.0   0.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0   
canceled          2.0  24.0   3.0  17.0  33.0  18.0  29.0  16.0  28.0  27.0   
created           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
invoiced          0.0  18.0  12.0  11.0   3.0  14.0  16.0  11.0   7.0  20.0   
processing        0.0   2.0   9.0  32.0  23.0  10.0  23.0  12.0  11.0  18.0   
shipped           1.0   8.0  16.0  21.0  45.0  49.0  55.0  47.0  56.0  41.0   
unavailable       0.0   7.0  10.0  45.0  32.0   9.0  31.0  24.0  52.0  32.0   

              ...                                                              
              ...                                                              
year          ...  2018                                                        
month         ...    1     2      3     4     5     6     7     8     9    10  
order_status  ...                                                              
approved      ...   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0  
canceled      ...  34.0  73.0   26.0  15.0  24.0  18.0  41.0  84.0  15.0  4.0  
created       ...   0.0   1.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0  
invoiced      ...  15.0   6.0   23.0  14.0  24.0   3.0  13.0  23.0   0.0  0.0  
processing    ...  29.0   6.0    9.0   8.0   6.0   0.0   1.0   0.0   0.0  0.0  
shipped       ...  74.0  57.0  133.0  99.0  54.0  43.0  60.0  47.0   1.0  0.0  
unavailable   ...  48.0  30.0   17.0   5.0  16.0   4.0  18.0   7.0   0.0  0.0  

[7 rows x 24 columns]

In [14]:
# считаю количество заказов каждом месяце, каждого года
orders_2_groupby = orders_2.groupby(['order_status','month','year'], as_index= False).order_id.count()

In [15]:
# считаю ср количество заказов в месяц
orders_2_groupby_2 = orders_2_groupby.groupby(['order_status','month']).agg({'order_id':'mean'})

In [16]:
#ДОП ТАБЛИЦА К ОТВЕТУ
# сводная таблица со ср колвом в мес
ex_2_2=pd.pivot_table(orders_2_groupby_2,
               index=["order_status"],
               values=["order_id"],
               columns= ["month"],
               aggfunc=[np.sum]).fillna(0)
ex_2_2

sum                                                       \
             order_id                                                        
month              1     2     3     4     5     6     7     8          9    
order_status                                                                 
approved          0.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0   0.000000   
canceled         18.5  45.0  29.5  16.5  26.5  17.0  34.5  55.5  12.333333   
created           0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.000000   
invoiced         13.5   8.5  13.0  14.0  20.0   7.0  10.0  21.5  17.000000   
processing       19.0  19.0  16.0   9.0  14.5  12.0   6.0  18.0  22.000000   
shipped          45.0  39.0  89.0  74.0  54.5  45.0  58.0  44.0  13.333333   
unavailable      29.0  37.5  24.5   7.0  23.5  14.0  35.0  19.5  38.000000   

                                
                                
month           10    11    12  
order_status                    
approved       0.0   0.0   0.0  
canceled      18.0  37.0  11.0  
created        0.0   2.0   2.0  
invoiced      17.0  35.0  13.0  
processing    11.0  25.0  35.0  
shipped       20.5  72.0  57.0  
unavailable   32.5  84.0  42.0

In [17]:
# считаю ср количество заказов в МЕСЯЦ (то есть, средняя, на среднее в месяц)
orders_2_groupby_2_2 = orders_2_groupby.groupby(['order_status','month'], as_index= False).agg({'order_id':'mean'})

In [18]:
orders_not_deliv = orders_2_groupby_2_2.groupby(['order_status']).agg({'order_id':'mean'}).round(1).sort_values('order_id').\
rename(columns ={'order_id':'avg_orders'}).reset_index()

In [19]:
# ВНИМАНИЕ ОТВЕТ
# Сколько заказов в месяц в среднем не доставляется (то есть не равно "Достпавлен") 
orders_not_deliv

,order_status,avg_orders
0,approved,1.0
1,created,1.7
2,invoiced,15.8
3,processing,17.2
4,canceled,26.8
5,unavailable,32.2
6,shipped,50.9


### Задача 3. По каждому товару определить, в какой день недели товар чаще всего покупается?
Под покупкой я подразумеваю ОПЛАЧЕННЫЙ (есть дата оплаты) и ДОСТАВЛЕННЫЙ (статус = delivered) заказ.  

In [20]:
order_by_items = order_items.merge(orders, how = 'left', on = 'order_id')
order_by_items.shape

(112650, 14)

In [21]:
order_by_items['order_approved_at']=pd.to_datetime(order_by_items['order_approved_at'])
order_by_items.dropna(subset=['order_approved_at'], inplace=True) # удаляю строки с пустой датой платежа
order_by_items['order_approved_at'].isnull().sum() # проверяю колво строк с пустой датой платежа
order_by_items['day_of_week'] = order_by_items['order_approved_at'].dt.weekday+1 #добавляю день недели
order_by_items=order_by_items.query("order_status == 'delivered'")

In [22]:
order_by_items.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,day_of_week
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,3
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,3


In [23]:
# ВНИМАНИЕ ОТВЕТ
order_by_items_gr= order_by_items.groupby(['product_id','day_of_week'], as_index = False)\
.order_id.count()\
.sort_values(['product_id','order_id'], ascending = [True, False])\
.groupby('product_id').head(1)
#!!!оставляем только дни где продажа была > 1 шт (понимаю что тем самым удалим часть товара)
order_by_items_gr=order_by_items_gr.query("order_id > 1") 
order_by_items_gr.sort_values('order_id', ascending = False)

,product_id,day_of_week,order_id
40186,aca2eb7d00ea1a7b8ebd4e68314663af,2,119
19705,53b36df67ebb7c41585e8d54d6772e08,2,104
15605,422879e10f46682990de24d770e7f83d,2,89
12792,368c6c730842d78016ad823897a372db,4,80
35774,99a4788cb24856965c36a24e339b6058,2,78
...,...,...,...
26021,6ed58ce8373b3d0c7704df9daf21e4d8,6,2
26012,6ed0ed10d62b45f3de46933b0b2989a6,1,2
26011,6eca85eeeb222dea025ec02b288a09a8,4,2
26009,6ec3c050cedb96661c07a891ee359146,6,2


### Задача 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?   
Не стоит забывать, что внутри месяца может быть не целое количество недель.   
Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.  

In [24]:
ord_by_cust = orders.merge(customers[['customer_unique_id','customer_id' ]], how = 'left', on = 'customer_id')\
.query("order_status =='delivered'")
ord_by_cust['order_purchase_timestamp']=pd.to_datetime(ord_by_cust['order_purchase_timestamp']) # формат даты создания заказа
ord_by_cust['year'] = ord_by_cust['order_purchase_timestamp'].dt.year
ord_by_cust['month'] = ord_by_cust['order_purchase_timestamp'].dt.month
#ord_by_cust['week'] = ord_by_cust['order_purchase_timestamp'].dt.week
ord_by_cust.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,year,month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,2017,10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,2018,7


In [25]:
# создаю df с колво недель в месяце
df_days = {'month': [1, 2,3,4,5,6,7,8,9,10,11,12], 'days_by_week': [4.429, 4 , 4.429, 4.286, 4.429, 4.286, 4.429, 4.429, 4.286, 4.429, 4.286,4.429]} 
df_days = pd.DataFrame(df_days) 
df_days.head(2)

,month,days_by_week
0,1,4.429
1,2,4.000


In [26]:
ord_by_cust_by_month = ord_by_cust.groupby(['customer_unique_id','year','month'], as_index = False)\
.order_id.count()\
.rename(columns = {'order_id':'quant_orders'}).sort_values('quant_orders')
ord_by_cust_by_month.head(2)

,customer_unique_id,year,month,quant_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,2018,5,1
63267,a9ee0ffb5493291751591e53992b1349,2017,12,1


In [27]:
#добавляю колво недель в каждом месяце для определения ср колва покупок
ord_by_cust_by_month_2= ord_by_cust_by_month.merge(df_days, how = 'left', on ='month')
ord_by_cust_by_month_2.head(2)

,customer_unique_id,year,month,quant_orders,days_by_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018,5,1,4.429
1,a9ee0ffb5493291751591e53992b1349,2017,12,1,4.429


In [198]:
# ВНИМАНИЕ ОТВЕТ
ord_by_cust_by_month_2['ord_per_week'] = ord_by_cust_by_month_2['quant_orders'] / ord_by_cust_by_month_2['days_by_week']
ord_per_week = ord_by_cust_by_month_2.groupby(['customer_unique_id','year','month'],as_index= False).agg({'ord_per_week':'mean'}).sort_values('customer_unique_id')
ord_per_week

,customer_unique_id,year,month,ord_per_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018,5,0.225785
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018,5,0.225785
2,0000f46a3911fa3c0805444483337064,2017,3,0.225785
3,0000f6ccb0745a6a4b88665a16c9f078,2017,10,0.225785
4,0004aac84e0df4da2b147fca70cf8255,2017,11,0.233318
...,...,...,...,...
95189,fffcf5a5ff07b0908bd4e2dbc735a684,2017,6,0.233318
95190,fffea47cd6d3cc0a88bd621562a9d061,2017,12,0.225785
95191,ffff371b4d645b6ecea244b27531430a,2017,2,0.250000
95192,ffff5962728ec6157033ef9805bacc48,2018,5,0.225785


In [199]:
# в дополнению к ответу определил сколько покупателей относится к каждой группе по Среднему колву заказов
avg_cust_ord = ord_per_week.groupby('ord_per_week').customer_unique_id.count().reset_index()
avg_cust_ord.rename(columns={'ord_per_week':'ср. заказов в неделю', 'customer_unique_id':'колво покупателей'})

,ср. заказов в неделю,колво покупателей
0,0.225785,57022
1,0.233318,29085
2,0.250000,7864
3,0.451569,692
4,0.466636,323
5,0.500000,157
6,0.677354,25
7,0.699953,12
8,0.750000,6
9,0.903138,2


### Задача 5. Коргортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. 

In [30]:
cust_by_orders = customers.merge(orders)
cust_by_orders.shape

(99441, 12)

In [31]:
#для расчетов как и в пред задание буду брать ОПЛАЧЕННЫЙ и ДОСТАВЛЕННЫЙ заказ
cust_by_orders['order_approved_at']=pd.to_datetime(cust_by_orders['order_approved_at'])
cust_by_orders.dropna(subset=['order_approved_at'], inplace=True) # удаляю строки с пустой датой платежа
cust_by_orders['order_approved_at'].isnull().sum() # проверяю колво строк с пустой датой платежа
cust_by_orders=cust_by_orders.query("order_status == 'delivered'")
cust_by_orders.shape

(96464, 12)

In [32]:
#создаю столбец с датой первой покупки
cust_by_orders['cog_analys'] = cust_by_orders.groupby('customer_unique_id')['order_approved_at'].transform('min')
cust_by_orders['cog_analys']=pd.to_datetime(cust_by_orders['cog_analys']) #привожу в нормальный формат
cust_by_orders.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,cog_analys
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05-16 15:22:12
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,2018-01-12 20:58:32


In [33]:
#создаю столбец с датой и месяцем
cust_by_orders.order_approved_at = cust_by_orders.order_approved_at.dt.strftime('%Y-%m')
cust_by_orders.cog_analys = cust_by_orders.cog_analys.dt.strftime('%Y-%m')

In [34]:
cust_by_orders.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,cog_analys
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,2018-01


In [35]:
#считаю количество уникальн.id
cust_by_orders_2 = cust_by_orders\
.query("'2017-01' <= cog_analys <= '2017-12'")\
.groupby(['cog_analys', 'order_approved_at'], as_index=False)\
.agg({'customer_unique_id':'nunique'})

In [36]:
#превращаем в номер месяца
cust_by_orders_2["order_approved_at"] = pd.to_datetime(cust_by_orders_2["order_approved_at"]).dt.to_period("m")
cust_by_orders_2["cog_analys"] = pd.to_datetime(cust_by_orders_2["cog_analys"]).dt.to_period("m")

In [37]:
cust_by_orders_2

,cog_analys,order_approved_at,customer_unique_id
0,2017-01,2017-01,683
1,2017-01,2017-02,3
2,2017-01,2017-03,2
3,2017-01,2017-04,1
4,2017-01,2017-05,3
...,...,...,...
166,2017-12,2018-04,13
167,2017-12,2018-05,11
168,2017-12,2018-06,9
169,2017-12,2018-07,1


In [38]:
cust_by_orders_2['period'] = (cust_by_orders_2.order_approved_at - cust_by_orders_2.cog_analys).apply(attrgetter('n'))

In [39]:
cust_by_orders_2

,cog_analys,order_approved_at,customer_unique_id,period
0,2017-01,2017-01,683,0
1,2017-01,2017-02,3,1
2,2017-01,2017-03,2,2
3,2017-01,2017-04,1,3
4,2017-01,2017-05,3,4
...,...,...,...,...
166,2017-12,2018-04,13,4
167,2017-12,2018-05,11,5
168,2017-12,2018-06,9,6
169,2017-12,2018-07,1,7


In [40]:
cust_by_orders_pivot = cust_by_orders_2.pivot(index='cog_analys', columns = 'period', values = 'customer_unique_id')
cust_by_orders_pivot

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
cog_analys,,,,,,,,,,,,,,,,,,,
2017-01,683.0,3.0,2.0,1.0,3.0,1.0,3.0,1.0,1.0,NaN,2.0,2.0,5.0,3.0,1.0,1.0,2.0,3.0,1.0
2017-02,1612.0,3.0,4.0,3.0,7.0,2.0,4.0,3.0,1.0,3.0,2.0,5.0,2.0,3.0,2.0,1.0,1.0,3.0,NaN
2017-03,2511.0,11.0,9.0,9.0,9.0,4.0,4.0,8.0,7.0,3.0,9.0,3.0,5.0,3.0,4.0,6.0,2.0,3.0,NaN
2017-04,2232.0,14.0,5.0,4.0,5.0,6.0,8.0,6.0,8.0,4.0,5.0,2.0,1.0,1.0,2.0,2.0,3.0,NaN,NaN
2017-05,3453.0,18.0,16.0,10.0,10.0,11.0,14.0,5.0,9.0,9.0,10.0,12.0,6.0,2.0,6.0,7.0,NaN,NaN,NaN
2017-06,3044.0,15.0,12.0,13.0,9.0,11.0,11.0,7.0,4.0,6.0,9.0,12.0,5.0,5.0,7.0,NaN,NaN,NaN,NaN
2017-07,3709.0,20.0,12.0,7.0,13.0,7.0,12.0,4.0,7.0,9.0,8.0,11.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN
2017-08,4081.0,28.0,15.0,11.0,15.0,21.0,11.0,11.0,5.0,7.0,10.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-09,4025.0,27.0,20.0,13.0,17.0,11.0,9.0,9.0,12.0,7.0,10.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# ВНИМАНИЕ ОТВЕТ
print ("Ответ:", cust_by_orders_pivot.div(cust_by_orders_pivot.iloc[:,0], axis = 0)[2].idxmax())

Ответ: 2017-09


### Задача 6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. 
Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

In [165]:
df_7 = cust_by_orders.merge(order_items)
df_7 = df_7.dropna()
df_7.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88


In [166]:
# нахожу дату последней покупки (на основании неё можно будет рассчитать количество дней прошедших с этого момента, т.е Recency)
df_7.order_approved_at = pd.to_datetime(df_7.order_approved_at)
last_day = df_7.order_approved_at.max()
last_day

Timestamp('2018-08-29 15:10:26')

In [167]:
rfm = df_7\
    .groupby('customer_unique_id')\
    .agg({'order_approved_at': lambda x: (last_day - x.max()).days, 'product_id': lambda x: len(x), 'price': lambda x: x.sum()})\
    .rename(columns={'order_approved_at': 'recency', 'product_id': 'frequency', 'price': 'monetary'})

In [168]:
rfm.head(2)

,recency,frequency,monetary
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,111,1,129.9
0000b849f77a49e4a4ce2b2a4ca5be3f,113,1,18.9


In [169]:
# Использую метод квинтилей, разделяя каждый признак на группы, содержащие 20% выборок 
quintiles = rfm[['recency', 'frequency', 'monetary']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'recency': {0.2: 91.0, 0.4: 176.0, 0.6: 267.0, 0.8: 381.0},
 'frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'monetary': {0.2: 39.9, 0.4: 69.9, 0.6: 109.9, 0.8: 179.9}}

In [170]:
# Присвоение баллов от 1 до 5
# Более высокие значения лучше подходят для частоты и денежного эквивалента, в то время как более низкие значения лучше подходят для новизны

def r_score(x):
    if x <= quintiles['recency'][.2]:
        return 5
    elif x <= quintiles['recency'][.4]:
        return 4
    elif x <= quintiles['recency'][.6]:
        return 3
    elif x <= quintiles['recency'][.8]:
        return 2
    else:
        return 1

In [171]:
def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.4]:
        return 2
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 4
    else:
        return 5

In [172]:
# Каждому клиенту мы присваиваем баллы R, F и M
rfm['r'] = rfm['recency'].apply(lambda x: r_score(x))
rfm['f'] = rfm['frequency'].apply(lambda x: fm_score(x, 'frequency'))
rfm['m'] = rfm['monetary'].apply(lambda x: fm_score(x, 'monetary'))
rfm

,recency,frequency,monetary,r,f,m
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,111,1,129.90,4,1,4
0000b849f77a49e4a4ce2b2a4ca5be3f,113,1,18.90,4,1,1
0000f46a3911fa3c0805444483337064,536,1,69.00,1,1,2
0000f6ccb0745a6a4b88665a16c9f078,320,1,25.99,2,1,1
0004aac84e0df4da2b147fca70cf8255,287,1,180.00,2,1,5
...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,446,2,1570.00,1,5,5
fffea47cd6d3cc0a88bd621562a9d061,261,1,64.89,3,1,2
ffff371b4d645b6ecea244b27531430a,567,1,89.90,1,1,3


In [173]:
rfm['rfm_score'] = rfm['r'].map(str) + rfm['f'].map(str) + rfm['m'].map(str)
rfm['rfm_score_sum'] = rfm['r'] + rfm['f']+ rfm['m']
rfm.head()

,recency,frequency,monetary,r,f,m,rfm_score,rfm_score_sum
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,111,1,129.90,4,1,4,414,9
0000b849f77a49e4a4ce2b2a4ca5be3f,113,1,18.90,4,1,1,411,6
0000f46a3911fa3c0805444483337064,536,1,69.00,1,1,2,112,4
0000f6ccb0745a6a4b88665a16c9f078,320,1,25.99,2,1,1,211,4
0004aac84e0df4da2b147fca70cf8255,287,1,180.00,2,1,5,215,8


In [178]:
rfm_ttl= rfm.groupby('rfm_score', as_index = False).recency.count().rename(columns={'recency':'count_customers'})
rfm_ttl['customers_share'] = (rfm_ttl.count_customers / 93342*100).round(2)
rfm_ttl.sort_values('rfm_score', ascending = False)

,rfm_score,count_customers,customers_share
49,555,952,1.02
48,554,537,0.58
47,553,417,0.45
46,552,258,0.28
45,551,141,0.15
44,515,2760,2.96
43,514,3374,3.61
42,513,3055,3.27
41,512,3520,3.77
40,511,3756,4.02


#### ВНИМАНИЕ ОТВЕТ:
Проанализировов таблицу с выводом выше, можно определить, что клиентов имеющих 5 или 4 баллов по всем категориям **2502** что составляет **2,68%** от всей клиентской массы.

In [191]:
rfm_ttl.query("rfm_score in ('555', '544', '545' , '455' , '444' , '454' ,'445')")\
.agg({'count_customers':'sum','customers_share':'sum' })

count_customers    2502.00
customers_share       2.68
dtype: float64

Также определим антирейтинг клиентов имеющих 1 или 2 балла по всем категориям **14861** что составляет **15,92%** от всей клиентской массы

In [192]:
rfm_ttl.query("rfm_score in ('111', '112', '212' , '222' , '211' , '212' ,'112')")\
.agg({'count_customers':'sum','customers_share':'sum' })

count_customers    14861.00
customers_share       15.92
dtype: float64